In [1]:
import pandas as pd
import numpy as np
import os
import gc
import time

cwd = os.getcwd()
os.chdir(cwd+'/../../../data/output csv files')

ratings = pd.read_csv('finalratings.csv')
finalbooks = pd.read_csv('finalbooks.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

os.chdir(cwd)

In [2]:
# Sort all books based on Popularity

# groupbooks by newbookid and get sum of ratings_count and average_ratings columns and sort them based on ratings_count in descending order and reset index
popular = finalbooks.groupby('newbookid')['ratings_count','average_rating'].sum().sort_values(by = 'ratings_count',ascending=False).reset_index()
popular['predicted_rating'] = np.round(popular['average_rating'])
popular

<ipython-input-2-83c97de9a246>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  popular = finalbooks.groupby('newbookid')['ratings_count','average_rating'].sum().sort_values(by = 'ratings_count',ascending=False).reset_index()


,newbookid,ratings_count,average_rating,predicted_rating
0,1,4780653,4.34,4.0
1,2,4602479,4.44,4.0
2,3,3866839,3.57,4.0
3,4,3198671,4.25,4.0
4,5,2683664,3.89,4.0
...,...,...,...,...
7995,7322,4886,3.89,4.0
7996,7153,4842,3.93,4.0
7997,7501,4528,4.29,4.0
7998,7807,4281,3.73,4.0


In [3]:
popular.to_csv('popular_books.csv',index=False)

In [4]:
useronly = ratings['newuser_id'].unique()
bookonly = popular['newbookid'].tolist()
top_predictions = pd.DataFrame({'newuser_id':pd.Series([], dtype='int'), 'newbookid':pd.Series([], dtype='int')})

# for i in range(len(useronly)):
#     all_books = bookonly
#     read_books = train.loc[ratings.newuser_id == i+1]['newbookid'].tolist()
#     rem_books = [i for i in bookonly if i not in read_books]
#     top_10=rem_books[:10]
#     predictions.append(top_10)
#     if (i+1)%1000 == 0: print("done: ", i+1)

start = time.time()
for i in range(len(useronly)):
    all_books = bookonly
    top_10=[]
    read_books = train.loc[ratings.newuser_id == i+1]['newbookid'].tolist()
    count=0
    k=0
    while(count<10):
        if all_books[k] not in read_books:
            top_predictions = top_predictions.append({'newuser_id': i+1, 'newbookid': all_books[k]}, ignore_index = True) 
            count=count+1
        k=k+1
    if (i+1)%1000 == 0: print("done: ", i+1)
end = time.time()

print("Completed in %dseconds"%(end-start))
top_predictions

done:  1000
done:  2000
done:  3000
done:  4000
done:  5000
done:  6000
done:  7000
done:  8000
done:  9000
done:  10000
done:  11000
done:  12000
done:  13000
done:  14000
done:  15000
Completed in 495seconds


,newuser_id,newbookid
0,1,2
1,1,3
2,1,5
3,1,8
4,1,9
...,...,...
149995,15000,6
149996,15000,7
149997,15000,8
149998,15000,10


In [5]:
top_predictions = top_predictions.merge(popular[['newbookid', 'ratings_count', 'average_rating']], how='left', on='newbookid')
top_predictions.to_csv("top_10_predictions_for_each_user_by_popularity.csv", index=False)
del top_predictions
gc.collect()

212